In [11]:
import random
import pandas as pd

# Константы для генерации данных
NUM_ITEMS = 100
NUM_TRANSACTIONS = 100000
NUM_RULES_HIGH_SUPPORT = 3
NUM_RULES_MEDIUM_SUPPORT = 5
NUM_RULES_LOW_SUPPORT = 10

# Вспомогательная функция для генерации правил
def generate_rules(num_rules, itemset_size):
    rules = []
    for _ in range(num_rules):
        items = random.sample(range(1, NUM_ITEMS + 1), itemset_size)
        rules.append(items)
    return rules

# Генерация правил разного уровня поддержки
high_support_rules = generate_rules(NUM_RULES_HIGH_SUPPORT, 2)
medium_support_rules = generate_rules(NUM_RULES_MEDIUM_SUPPORT, 3)
low_support_rules = generate_rules(NUM_RULES_LOW_SUPPORT, 4)

# Вспомогательная функция для создания транзакции
def create_transaction():
    transaction = set()
    # Добавление правил с вероятностью, соответствующей уровню поддержки
    if random.random() < 0.1:  # Высокая поддержка
        transaction.update(random.choice(high_support_rules))
    if random.random() < 0.05: # Средняя поддержка
        transaction.update(random.choice(medium_support_rules))
    if random.random() < 0.01: # Низкая поддержка
        transaction.update(random.choice(low_support_rules))
    # Добавление случайных элементов
    num_additional_items = random.randint(0, 5)  # До 5 дополнительных элементов
    additional_items = random.sample(range(1, NUM_ITEMS + 1), num_additional_items)
    transaction.update(additional_items)
    return list(transaction)

# Генерация транзакций
transactions = [create_transaction() for _ in range(NUM_TRANSACTIONS)]

In [13]:
def generate_candidates(prev_itemsets, k):
    """Генерация кандидатов для нового уровня из предыдущего набора частых наборов"""
    candidates = set()
    for a in prev_itemsets:
        for b in prev_itemsets:
            union = a.union(b)
            if len(union) == k:
                candidates.add(union)
    return candidates

def apriori(transactions, min_support):
    """Реализация алгоритма Apriori для нахождения частых наборов"""
    # Первый шаг: подсчёт одноэлементных частых наборов
    item_support = {}
    for transaction in transactions:
        for item in transaction:
            itemset = frozenset([item])
            if itemset in item_support:
                item_support[itemset] += 1
            else:
                item_support[itemset] = 1

    n_transactions = len(transactions)
    itemsets = [itemset for itemset, count in item_support.items() if count / n_transactions >= min_support]

    # Хранение всех частых наборов
    frequent_itemsets = list(itemsets)
    k = 2

    # Итерация по уровням
    while itemsets:
        candidates = generate_candidates(itemsets, k)
        support_counts = defaultdict(int)
        for transaction in transactions:
            for candidate in candidates:
                if candidate.issubset(transaction):
                    support_counts[candidate] += 1

        # Фильтрация кандидатов с достаточной поддержкой
        itemsets = [itemset for itemset, count in support_counts.items() if count / n_transactions >= min_support]
        frequent_itemsets.extend(itemsets)
        k += 1

    return frequent_itemsets

# Пример использования
#transactions = load_transactions()  # Загрузка транзакций из файла
min_support = 0.1  # Минимальный уровень поддержки
frequent_itemsets = apriori(transactions, min_support)
print("Частые наборы элементов:")
for itemset in frequent_itemsets:
    print(itemset)

Частые наборы элементов:
